# **Group 14 Apply AI Project: _________________________________**                                   
Members: Torrence Barbour, Satwik Nijampudi, Shanthan Gunti, and Ryan Edwards

Given a set of posts found on Reddit marked with if the user had depression or not, can we train a model to detect depression in a social media user?

In [79]:
#We need to import some libraries to do this, including some big ones like numpy, pandas, and matplotlib
import os
import time
import re
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt


import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

## Further Cleaning our data                                                                                                  
We got our data from a kaggle dataset: https://www.kaggle.com/datasets/infamouscoder/depression-reddit-cleaned and it came pre-cleaned, however we want to further clean this dataset to suit our intended purposes (like lemmatization, removing filler words, and more). We will do this now.

In [9]:
#First, we will need to bring in our dataset. We can use numpy for this
text = pd.read_csv('depression_dataset_reddit_cleaned.csv')
text.head()

,clean_text,is_depression
0,we understand that most people who reply immed...,1
1,welcome to r depression s check in post a plac...,1
2,anyone else instead of sleeping more when depr...,1
3,i ve kind of stuffed around a lot in my life d...,1
4,sleep is my greatest and most comforting escap...,1


In [11]:
#Next, we are going to need a tool to tokenize, lemmatize, and manage our data. We will use spacy
import spacy
!python3 -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 8.6 MB/s eta 0:00:000m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
#Just to confirm, lets print out one of the posts
print(text.iloc[2])

clean_text       anyone else instead of sleeping more when depr...
is_depression                                                    1
Name: 2, dtype: object


Now that we have some tools, our dataset, and another tool used for NLP installed, we need to start preparing our data so it can be used. The first step in this is something called tokenization. Essentially, each word (or even punctuation for that matter) in a sentence plays a role and it can be looked at as its own entity. We want to break each of these sentences apart into 'tokens' so they are more processable.

In [24]:
#Lets tokenize the sample sentence from before as an example
sample_Tokens = nlp(text.iloc[2]['clean_text'])
for part in sample_Tokens: 
    print(part.text)

anyone
else
instead
of
sleeping
more
when
depressed
stay
up
all
night
to
avoid
the
next
day
from
coming
sooner
may
be
the
social
anxiety
in
me
but
life
is
so
much
more
peaceful
when
everyone
else
is
asleep
and
not
expecting
thing
of
you


On top of tokenization, there is also lemmatization. A lemma in linguistics is essentially the core component of a word, before conjugation and the like occurs. This is very important because it essentially determines what role the word will play in the sentence. Will it be a noun? A verb? Lets see a few examples of this in code

In [38]:
#The .lemma_ command in spacy will output the lemma of the word it was called with. This will come in handy
print('The lemma of', sample_Tokens[18], 'is', sample_Tokens[18].lemma_)
print('The lemma of', sample_Tokens[4], 'is', sample_Tokens[4].lemma_)
print('The lemma of', sample_Tokens[19], 'is', sample_Tokens[19].lemma_)

The lemma of coming is come
The lemma of sleeping is sleep
The lemma of sooner is soon


In [46]:
#The .pos_ command will output the part of speech the word plays in a sentence.
print('The part of speech of the word', sample_Tokens[18].lemma_, 'is', sample_Tokens[18].pos_)
print('The part of speech of the word', sample_Tokens[19].lemma_, 'is', sample_Tokens[19].pos_)
print('The part of speech of the word', sample_Tokens[24].lemma_, 'is', sample_Tokens[24].pos_)

The part of speech of the word come is VERB
The part of speech of the word soon is ADV
The part of speech of the word anxiety is NOUN


Now, lets use this information in order to get our dataset cleaned to the utmost. The only things left should be easily recognizable by future tools.

In [2]:
#Create an empty array to store the cleaned posts in
cleaned_Posts = []

#Parse through every post
for uncleaned_Post in range(len(text)):
    to_Clean = text.iloc[uncleaned_Post].copy() #We need to make a copy or else we will mess with the original data
    
    #Lets make all the tokens lowercase so capitalization is no longer a concern (We can use PCRE regular expressions)
    to_Clean_More = re.sub("[^A-Za-z']+", ' ', to_Clean['clean_text'].lower()
    
    #Tokenize and Lemmatize the posts now
    to_Clean_Even_More = nlp(to_Clean_More)
    absurdly_Clean_Now = [word.lemma_ for word in to_Clean_Even_More if ((not word.is_stop) or (' ' in word.text))]
    
    #If there is any text left in absurdly_Clean_Now the add it to the new column
    if len(absurdly_Clean_Now) > 1:
        to_Clean['really_Clean'] = ' '.join(absurdly_Clean_Now)
    
    #Now we just append the row to our cleaned_Posts array
    cleaned_Posts.append(to_Clean)
    
#Create a new data frame with the new column in it
clean_Text = pd.DataFrame(cleaned_Posts)

SyntaxError: EOL while scanning string literal (3510832948.py, line 9)

In [56]:
#If we save the dataset now we can save a lot of time later, so lets do that
clean_Text.to_csv('cleaned_depression_dataset.csv')
clean_Text.head()
print(clean_Text.iloc[0])

clean_text       we understand that most people who reply immed...
is_depression                                                    1
really_Clean       w e u n d e r s t n d t h t m o s t p e o p ...
Name: 0, dtype: object


This has been a good introduction to how NLP cleans data, but for the sake of efficiency, we are going to use a library known as HuggingFace to get our word vectors

In [2]:
import transformers as hug

With HuggingFace, we can use a model known as BERT which has been trained on tons of the English language in order to tokenize our sentences and turn them into word vectors in just a few commands.

In [15]:
#This is adding in the BERT tokenizer which we will use to tokenize the posts
theTokenizer = hug.AutoTokenizer.from_pretrained('bert-base-uncased')

In [16]:
#This is adding in the BERT model which we will use to get the word vectors
theModel = hug.AutoModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Now, lets get the word vectors for our posts

In [68]:
posts = pd.read_csv('depression_dataset_reddit_cleaned.csv')
words = posts.copy()
type(words['clean_text'][1])

str

In [ ]:
word_Vectors = []

#Parse through every post
for x in tqdm(range(len(posts))):
    to_Clean = words['clean_text'][x]
    inputs = theTokenizer(to_Clean, padding='max_length',truncation = True, max_length = 512, return_tensors="pt")
    output = theModel(**inputs)
    
    word_Vectors.append(output[1])
    
#Create a new data frame with the new column in it
vectors = pd.DataFrame(cleaned_Posts)
vectors.head()

  0%|          | 0/7731 [00:00<?, ?it/s]

And of course, since we do not want to run this again and again, let's save the output to a .csv file in order to keep the data more permanently

In [1]:
vectors.to_csv('cleaned_depression_dataset.csv')

NameError: name 'vectors' is not defined